In [0]:
# If implemented in the same way as the other models, maximum accuracy 74% and overfitting after 15 epochs. (with data generator)
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import *
import matplotlib.pyplot as plt
import numpy as np
from keras import regularizers
from keras.callbacks import LearningRateScheduler

In [0]:
 def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
num_classes = 10

def get_model(x_train,y_train,x_test,y_test):
  
  batch_size = 64
  epochs = 125
  data_augmentation = True
  weight_decay = 1e-4

  model = Sequential()                                                                                 # base model
  model.add(Conv2D(32, kernel_size=3, strides=(1, 1), input_shape =(32,32,3), activation ='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay) ))     # as the image is in grayscale the shape is 28,28,1 (if rgb = (28,28,3))
                                                                                                       # Conv2D is used for spatial convolution over images, 64 feature maps
  model.add(BatchNormalization())
  model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size = (2,2))) 
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.4))
 
  model.add(Flatten())
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  print("\n Model Summary \n")
  model.summary()
  
 
  # initiate RMSprop optimizer
  opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
  model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
  
  if not data_augmentation:
      print('Not using data augmentation.')
      model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                shuffle=True)
  else:
      print('Using real-time data augmentation.')
      # This will do preprocessing and realtime data augmentation:
      datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          zca_epsilon=1e-06,  # epsilon for ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          # randomly shift images horizontally (fraction of total width)
          width_shift_range=0.1,
          # randomly shift images vertically (fraction of total height)
          height_shift_range=0.1,
          shear_range=0.,  # set range for random shear
          zoom_range=0.,  # set range for random zoom
          channel_shift_range=0.,  # set range for random channel shifts
          # set mode for filling points outside the input boundaries
          fill_mode='nearest',
          cval=0.,  # value used for fill_mode = "constant"
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False,  # randomly flip images
          # set rescaling factor (applied before any other transformation)
          rescale=None,
          # set function that will be applied on each input
          preprocessing_function=None,
          # image data format, either "channels_first" or "channels_last"
          data_format=None,
          # fraction of images reserved for validation (strictly between 0 and 1)
          validation_split=0.0)

      # Compute quantities required for feature-wise normalization
      # (std, mean, and principal components if ZCA whitening is applied).
      datagen.fit(x_train)

      # Fit the model on the batches generated by datagen.flow().
      history = model.fit_generator(datagen.flow(x_train, y_train,
                                       batch_size=batch_size),\
                          steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                          verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)], workers=4)
      
        print("\n Model Training \n")
 
      plt.plot(history.history['acc'])                                                                        # summarize history for accuracy
      plt.plot(history.history['val_acc'])
      plt.title('\n Model Accuracy \n')
      plt.ylabel('accuracy') 
      plt.xlabel('epoch')
      plt.legend(['train', 'test'], loc='upper left')
      plt.show()

      plt.plot(history.history['loss'])                                                                       # summarize history for loss
      plt.plot(history.history['val_loss'])
      plt.title('\n Model Loss \n')
      plt.ylabel('loss')
      plt.xlabel('epoch')
      plt.legend(['train', 'test'], loc='upper left')
      plt.show()
           
  return model

In [0]:
def train_model():
  
  # The data, split between train and test sets:
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  print('x_train shape:', x_train.shape)
  print('x_test shape:',x_test.shape)
  print(x_train.shape[0], 'train samples')
  print(x_test.shape[0], 'test samples')
  
  # Convert class vectors to binary class matrices.
  y_train = to_categorical(y_train, num_classes)
  y_test = to_categorical(y_test, num_classes)
  
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  
  mean = np.mean(x_train,axis=(0,1,2,3))
  std = np.std(x_train,axis=(0,1,2,3))
  x_train = (x_train-mean)/(std+1e-7)
  x_test = (x_test-mean)/(std+1e-7)

  print("\n Training model - \n")
  model = get_model(x_train,y_train,x_test,y_test)                # Function call for training
  y_test_pred = model.predict(x_test)                             # Testing
  
  print("\n Some Outputs\n")
  for i in range(10):                                             # Print first 10 test images with output classes 
    img = x_test[i].reshape((32,32,3))
    plt.imshow(img, cmap="Greys")
    plt.show()
    print("Target Value",np.argmax(y_test_pred[i]))
  
  acc = model.evaluate(x_test,y_test,verbose=2)                   # Evaluation of model
  print('\n TEST LOSS: ', acc[0])
  print('\n TEST ACCURACY: ',acc[1])
  
  return model

In [0]:
train_model()

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
50000 train samples
10000 test samples

 Training model - 


 Model Summary 

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 1